In [1]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [2]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [4]:
#Start classes
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.02 sec
Read 100 history items for RemoteEvent(ATDome, 0, allAxesInPosition)
Read 26 history items for RemoteEvent(ATDome, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATDome, 0, authList)
Read 81 history items for RemoteEvent(ATDome, 0, azimuthCommandedState)
Read 100 history items for RemoteEvent(ATDome, 0, azimuthInPosition)
Read 100 history items for RemoteEvent(ATDome, 0, azimuthState)
Read 100 history items for RemoteEvent(ATDome, 0, doorEncoderExtremes)
Read 21 history items for RemoteEvent(ATDome, 0, dropoutDoorCommandedState)
Read 7 history items for RemoteEvent(ATDome

[[None, None, None, None, None, None, None], [None, None, None, None]]

trajectory DDS read queue is filling: 36 of 100 elements
m1AirPressure DDS read queue is filling: 20 of 100 elements
mount_positions DDS read queue is filling: 27 of 100 elements
torqueDemand DDS read queue is filling: 36 of 100 elements
loadCell DDS read queue is filling: 20 of 100 elements
mountStatus DDS read queue is full (100 elements); data may be lost
nasymth_m3_mountMotorEncoders DDS read queue is filling: 36 of 100 elements
guidingAndOffsets DDS read queue is full (100 elements); data may be lost


In [ ]:
# enable components
await atcs.enable()
await latiss.enable()

In [ ]:
await latiss.rem.atcamera.cmd_start.set_start(timeout=30)

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.ENABLED)

In [ ]:
await latiss.rem.atarchiver.cmd_start.set_start(timeout=30)

In [ ]:
await salobj.set_summary_state(latiss.rem.atarchiver, salobj.State.ENABLED)

In [ ]:
# All components now enabled. - 18-Feb-2021 5:43 PM

In [ ]:
# Ensure we're using Nasmyth 2
await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [ ]:
await atcs.prepare_for_flatfield()

In [ ]:
await latiss.take_bias(1)

In [ ]:
# Take 50 biases seq # 7-56
# Added wait to stop killing the recent images
for i in range(50):
    await asyncio.sleep(2.0)
    await latiss.take_bias(1)

In [ ]:
# Take 10 10 second darks 57-66
await latiss.take_darks(10.0, 10)

In [ ]:
# Take 10 2 second flats 67-76
await latiss.take_flats(2.0, 10, filter='RG610', grating='empty_1')

In [ ]:
# Take flats for PTC 77-116
# Added wait to stop killing the recent images
for i in range(20):
    exp = 0.2 * float(i+1)
    await latiss.take_flats(exp, 1, filter='empty_1', grating='empty_1')
    if exp < 2.0:
        await asyncio.sleep(2.0)
    await latiss.take_flats(exp, 1, filter='empty_1', grating='empty_1')


In [7]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.ENABLED)

[<State.DISABLED: 1>, <State.ENABLED: 2>]

In [8]:
current_az = 90.0
current_el = 80.0
current_rot = 0.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

Sending command
Stop tracking.
Unknown tracking state: 9
Unknown tracking state: 10
In Position: True.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
Got True
Waiting for telescope to settle.
[Telescope] delta Alt = -000.001 deg; delta Az= +000.002 deg; delta N1 = -000.000 deg; delta N2 = +000.000 deg 
Axes in position.


In [13]:
# Take a sky flat to see how overexposed it is
await latiss.take_flats(0.1, 1, filter='empty_1', grating='empty_1')

Generating group_id
FLAT 0001 - 0001


array([2021021800120])

In [20]:
# Running sky flats as the sky darkens
for i in range(5):
    exp = 20.0
    await latiss.take_flats(exp*2.0, 1, filter='RG610', grating='empty_1')
    await asyncio.sleep(5.0)
    await latiss.take_flats(exp, 1, filter='empty_1', grating='empty_1')
    await asyncio.sleep(60.0)

Generating group_id
FLAT 0001 - 0001
Generating group_id
FLAT 0001 - 0001
Generating group_id
FLAT 0001 - 0001
Generating group_id
FLAT 0001 - 0001


CancelledError: 

logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 14 of 100 elements
logMessage DDS read queue is filling: 10 of 100 elements
target DDS read queue is filling: 19 of 100 elements
target python read queue is filling: 18 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 13 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 10 of 100 elements
target DDS read queue is filling: 20 of 100 elements
target python read queue is filling: 19 of 100 elements
target DDS read queue is filling: 20 of 100 elements
target python read queue is filling: 19 of 100 elements
logMessage DDS read queue is filling: 12 of 100 elements
logMessage DDS read queue is filling: 10 of 10

In [ ]:
await latiss.standby()

In [ ]:
await atcs.close_m1_vent()

In [ ]:
await atcs.close_m1_cover()

In [ ]:
await atcs.stop_tracking()

In [ ]:
# Putting everything back in standby.
await atcs.standby()

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply="test")

In [ ]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [ ]:
# Disable the dome
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.DISABLED, settingsToApply="test")
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.DISABLED)

In [ ]:
# Take event checking out the slew commands to test telescope only
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [ ]:
await atcs.rem.atdome.cmd_start.set_start(settingsToApply="test", timeout=30)

In [ ]:
# Re-enable the dome
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = True
atcs.check.atdometrajectory = True

In [ ]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.ENABLED, settingsToApply="test")

In [ ]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.ENABLED)